In [1]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## 1. Downloading the data

We'll use [the same NYC taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

In [2]:
df_01_2023 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet", engine="pyarrow")

In [3]:
df_02_2023 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet", engine="pyarrow")

In [4]:
print(f"Total Columns: {len(df_01_2023.columns)}")

Total Columns: 19


## 2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [5]:
df_01_2023.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [6]:
df_01_2023["duration"] = df_01_2023["tpep_dropoff_datetime"] - df_01_2023["tpep_pickup_datetime"]

In [7]:
df_01_2023["duration"] = df_01_2023["duration"].dt.total_seconds() / 60

In [8]:
df_01_2023["duration"].head()

0     8.433333
1     6.316667
2    12.750000
3     9.616667
4    10.833333
Name: duration, dtype: float64

In [9]:
print(f"January trips duration standard deviation: {df_01_2023["duration"].std()}")

January trips duration standard deviation: 42.59435124195458


## 3. Dropping outliers

Next, we need to check the distribution of the durationx`variable`. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [10]:
df_01_2023_without_outliers = df_01_2023[(df_01_2023["duration"] >= 1) & (df_01_2023["duration"] <= 60)].copy()

In [11]:
print(f"Fraction of the records:  % {(df_01_2023_without_outliers["duration"].count() / df_01_2023["duration"].count()) * 100}")

Fraction of the records:  % 98.1220282212598


## 4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [12]:
df_01_2023_without_outliers.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [13]:
df_01_2023_without_outliers[["PULocationID", "DOLocationID"]] = df_01_2023_without_outliers[["PULocationID", "DOLocationID"]].astype(str)  

In [14]:
dv = DictVectorizer()

train_dicts = df_01_2023_without_outliers[["PULocationID", "DOLocationID"]].to_dict(orient="records")

In [15]:
X_train = dv.fit_transform(train_dicts) 

In [16]:
print(f"Dimensionality of matrix (number of columns): {X_train.shape[1]}")

Dimensionality of matrix (number of columns): 515


## 5. Training a model

Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [17]:
target = "duration"

In [18]:
y_train = df_01_2023_without_outliers[target].values

In [19]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [20]:
y_pred = model.predict(X_train)

In [21]:
RMSE = mean_squared_error(y_train, y_pred, squared=False)

/home/samir/miniconda3/envs/mlops/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [22]:
print(f"RMSE: {RMSE}")

RMSE: 7.649261936284003


## 6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [23]:
# Computing duration
df_02_2023["duration"] = df_02_2023["tpep_dropoff_datetime"] - df_02_2023["tpep_pickup_datetime"]
df_02_2023["duration"] = df_02_2023["duration"].dt.total_seconds() / 60

In [24]:
# outliers
df_02_2023_without_outliers = df_02_2023[(df_02_2023["duration"] >= 1) & (df_02_2023["duration"] <= 60)].copy()

In [25]:
# One-hot encoding
df_02_2023_without_outliers[["PULocationID", "DOLocationID"]] = df_02_2023_without_outliers[["PULocationID", "DOLocationID"]].astype(str)  
valildation_dicts = df_02_2023_without_outliers[["PULocationID", "DOLocationID"]].to_dict(orient="records")
X_validation = dv.transform(valildation_dicts) 

In [30]:
# model
y_validation = df_02_2023_without_outliers[target].values
y_prediction = model.predict(X_validation)
RMSE_validation = mean_squared_error(y_validation, y_prediction, squared=False)

/home/samir/miniconda3/envs/mlops/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [31]:
print(f"RMSE validation: {RMSE_validation}")

RMSE validation: 7.811818654341152
